In [18]:
!pip install gensim --upgrade
!pip install keras --upgrade
!pip install pandas --upgrade

                                              0.0/24.0 MB ? eta -:--:--
                                             0.0/24.0 MB 660.6 kB/s eta 0:00:37
                                             0.1/24.0 MB 657.6 kB/s eta 0:00:37
                                             0.2/24.0 MB 893.0 kB/s eta 0:00:27
                                              0.3/24.0 MB 1.1 MB/s eta 0:00:23
                                              0.4/24.0 MB 1.2 MB/s eta 0:00:21
                                              0.4/24.0 MB 1.2 MB/s eta 0:00:20
                                              0.6/24.0 MB 1.3 MB/s eta 0:00:19
     -                                        0.7/24.0 MB 1.5 MB/s eta 0:00:16
     -                                        1.0/24.0 MB 1.9 MB/s eta 0:00:13
     --                                       1.4/24.0 MB 2.2 MB/s eta 0:00:11
     --                                       1.7/24.0 MB 2.5 MB/s eta 0:00:09
     ---                                      2.0/24.0 M

                                              0.0/1.7 MB ? eta -:--:--
                                              0.0/1.7 MB 660.6 kB/s eta 0:00:03
     -                                        0.1/1.7 MB 657.6 kB/s eta 0:00:03
     ---                                      0.1/1.7 MB 774.0 kB/s eta 0:00:03
     -----                                    0.2/1.7 MB 1.1 MB/s eta 0:00:02
     --------                                 0.4/1.7 MB 1.2 MB/s eta 0:00:02
     -----------                              0.5/1.7 MB 1.3 MB/s eta 0:00:01
     -------------                            0.6/1.7 MB 1.4 MB/s eta 0:00:01
     ------------------                       0.8/1.7 MB 1.6 MB/s eta 0:00:01
     ---------------------                    0.9/1.7 MB 1.6 MB/s eta 0:00:01
     ------------------------                 1.1/1.7 MB 1.7 MB/s eta 0:00:01
     ---------------------------              1.2/1.7 MB 1.7 MB/s eta 0:00:01
     -------------------------------          1.4/1.7 MB 1.8 MB/s

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\User\\anaconda3\\Lib\\site-packages\\~andas\\_libs\\algos.cp38-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [2]:
# Importing Packages
import warnings
warnings.filterwarnings("ignore") 
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
from wordcloud import WordCloud,STOPWORDS,ImageColorGenerator
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
import urllib.request
import re
import nltk
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from gensim.models import Word2Vec
from sklearn.linear_model import PassiveAggressiveClassifier,LogisticRegression
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout,Input,SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow import keras
from nltk.stem import WordNetLemmatizer


In [3]:
df = pd.read_csv("C:/Users/User/Downloads/Twitter_Data.csv", encoding = 'latin-1', nrows=2000)

In [4]:
# Data Sample
df.sample(5)

,clean_text,category
1354,the able capable and innovative enough look in...,1
1357,but you questioned modi jis headgears giving t...,1
794,let anyone give single reason should not vote ...,1
1209,anpad ankitaise bolegaaadont surprise others t...,1
828,then shld have election between mms and modi i...,1


In [5]:
# Checking for NA Values
df.isnull().sum()

clean_text    1
category      0
dtype: int64

In [6]:
# Distribution of different classes in sentiment
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=["Total","Percentage"])
count_values_in_column(df,"category")

,Total,Percentage
category,,
1,868,43.4
0,706,35.3
-1,426,21.3


In [7]:
# Segrating based on different sentiments
df_negative = df[df["category"]==-1]
df_positive = df[df["category"]==1]
df_neutral = df[df["category"]==0]

In [8]:
# Stemming
ps = PorterStemmer()
# Initializing Lists
corpus = []
words = []
for i in range(0, len(df)):
    # Removing characters other than letters
    review = re.sub('[^a-zA-Z]', ' ', str(df["clean_text"][i]))
    # Lowering the case all the text
    review = review.lower()
    # Splitting into words
    review = review.split()
    # Applying Stemming
    stemmed = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    # Joining words
    review = ' '.join(stemmed)
    # Appending all tweets to a list after preprocessing
    corpus.append(review)
    # Appending all words for word embeddings
    words.append(stemmed)

In [9]:
# Corpus sample
corpus[1:10]

['talk nonsens continu drama vote modi',
 'say vote modi welcom bjp told rahul main campaign modi think modi relax',
 'ask support prefix chowkidar name modi great servic confus read crustal clear crass filthi nonsens see abus come chowkidar',
 'answer among power world leader today trump putin modi may',
 'kiya tho refresh maarkefir comment karo',
 'surat women perform yagna seek divin grace narendra modi becom',
 'come cabinet scholar like modi smriti hema time introspect',
 'upcom elect india saga go import pair look current modi lead govt elect deal brexit combin weekli look juici bear imho',
 'gandhi gay modi']

In [10]:
# Length 
print("Legth of Corpus:",len(corpus))

Legth of Corpus: 2000


In [11]:
# Updating created corpus in our dataframe
df["clean_text"] = corpus

In [12]:
# Dropping NA Values and resetting index
df = df.dropna()
df = df.reset_index()

In [13]:
# Checking for NA Values after corpus updations
df.isna().sum()

index         0
clean_text    0
category      0
dtype: int64

In [14]:
# Exporting stemmed sentences
df[["clean_text","category"]].to_csv("stemmed.csv",index = False)

In [15]:
# Loading the stemmed sentences
df_stemmed = pd.read_csv("stemmed.csv")
# Extracting corpus
corpus = list(df_stemmed["clean_text"])

In [16]:
# Applying TFIDF Vectorization
tfidf = TfidfVectorizer(max_features=5000,ngram_range=(1,3))
X_tfidf = tfidf.fit_transform(df["clean_text"]).toarray()

In [17]:
# Independent Variable
X = df_stemmed["clean_text"]
# Dependent Varible
Y=df_stemmed["category"]
df_tfidf = pd.DataFrame(X_tfidf,columns = tfidf.get_feature_names())
df_tfidf["output"] = Y
df_tfidf.head()

,aadhaar,aap,abe,abil,abl,abp,absolut,absolut hate,absolut hate congress,absolut right,...,yogi,yogi adityanath,your,youth,youth victim,youth victim demo,youtub,yr,zone,output
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1


In [18]:
# Train test Split
X_train_tfidf,X_test_tfidf,Y_train_tfidf,Y_test_tfidf = train_test_split(X_tfidf,Y,test_size=0.33,random_state = 27)

In [19]:
# Initializing Model
classfier_tfidf = MultinomialNB(alpha=0.1)
# Fitting data
classfier_tfidf.fit(X_train_tfidf,Y_train_tfidf)
# Prediction on test data
Y_pred_tfidf = classfier_tfidf.predict(X_test_tfidf)

In [20]:
# Initializing Model
logistic_tfidf = LogisticRegression(solver='liblinear')
# Fitting data
logistic_tfidf.fit(X_train_tfidf,Y_train_tfidf)
# Prediction on test data
Y_pred_logistic_tfidf = logistic_tfidf.predict(X_test_tfidf)

In [21]:
acc_log_tfidf = accuracy_score(Y_test_tfidf,Y_pred_logistic_tfidf)
classification_log_tfidf = classification_report(Y_test_tfidf,Y_pred_logistic_tfidf)
confusion_matrix_log_tfidf = confusion_matrix(Y_test_tfidf,Y_pred_logistic_tfidf)

In [22]:
print("For Logistic Regression: \n")
print(" \n Accuracy : ",acc_log_tfidf,"\n","Classification report \n",classification_log_tfidf,"\n","Confusion matrix \n",confusion_matrix_log_tfidf)

For Logistic Regression: 

 
 Accuracy :  0.5818181818181818 
 Classification report 
               precision    recall  f1-score   support

          -1       0.71      0.08      0.15       143
           0       0.69      0.55      0.61       243
           1       0.53      0.87      0.66       274

    accuracy                           0.58       660
   macro avg       0.64      0.50      0.47       660
weighted avg       0.63      0.58      0.53       660
 
 Confusion matrix 
 [[ 12  30 101]
 [  0 133 110]
 [  5  30 239]]


In [23]:
acc_tfidf = accuracy_score(Y_test_tfidf,Y_pred_tfidf)
classification_tfidf = classification_report(Y_test_tfidf,Y_pred_tfidf)
confusion_matrix_tfidf = confusion_matrix(Y_test_tfidf,Y_pred_tfidf)

In [24]:
print("For Mutinomial Naive Bayes: \n")
print(" \n Accuracy : ",acc_tfidf,"\n","Classification report \n",classification_tfidf,"\n","Confusion matrix \n",confusion_matrix_tfidf)

For Mutinomial Naive Bayes: 

 
 Accuracy :  0.55 
 Classification report 
               precision    recall  f1-score   support

          -1       0.42      0.36      0.39       143
           0       0.67      0.47      0.55       243
           1       0.54      0.72      0.62       274

    accuracy                           0.55       660
   macro avg       0.54      0.52      0.52       660
weighted avg       0.56      0.55      0.54       660
 
 Confusion matrix 
 [[ 52  20  71]
 [ 31 114  98]
 [ 40  37 197]]


In [25]:
# Cloning tweets
messages=X.copy()
# Setting parameter for padding and sequential modelling
voc_size = 5000
embedding_vector_features = 200
sent_length = 200
lstm_out = 128
# Tokenization of all words in the vocabulary for all tweets
onehot_repr=[one_hot(words,voc_size)for words in df["clean_text"]]
onehot_repr[1:3]

[[774, 2530, 401, 14, 464, 1238],
 [817, 464, 1238, 1976, 2026, 4759, 1166, 4894, 336, 1238, 869, 1238, 4018]]

In [26]:
# Applying Post Padding
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1238 3489 2942 ...    0    0    0]
 [ 774 2530  401 ...    0    0    0]
 [ 817  464 1238 ...    0    0    0]
 ...
 [2510  464 2262 ...    0    0    0]
 [  90 3126 1030 ...    0    0    0]
 [ 760 4635 1238 ...    0    0    0]]


In [27]:
# Changing Negative values to positive
Y = [2 if x == -1 else x for x in Y]

# Converting list to arrays
X_final=np.array(embedded_docs)
y_final=np.array(Y)
# Shape of X,Y
X_final.shape,y_final.shape

((2000, 200), (2000,))

In [31]:
# Train Test Split
X_train_embed, X_test_embed, Y_train_embed, Y_test_embed = train_test_split(X_final, y_final, test_size=0.33, random_state=27)
# Sequential Model
model = Sequential()

# Input layer
model.add(Input(shape=(None,)))

# Embedding layer
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))

# LSTM layer
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2)))

# Fully connected layer
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))

# Output layer
model.add(Dense(3, activation = 'softmax'))

# model.summary()
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 40)          200000    
                                                                 
 bidirectional_3 (Bidirectio  (None, 200)              112800    
 nal)                                                            
                                                                 
 dense_5 (Dense)             (None, 128)               25728     
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_6 (Dense)             (None, 64)                8256      
                                                                 
 dense_7 (Dense)             (None, 3)                 195       
                                                      

In [32]:
# Compling model and running
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(X_train_embed,Y_train_embed,validation_data=(X_test_embed,Y_test_embed),
                    epochs=2, batch_size=256,
                    verbose=1,shuffle=True)

Epoch 1/2
6/6 [==============================] - 13s 1s/step - loss: 1.0871 - accuracy: 0.4246 - val_loss: 1.0671 - val_accuracy: 0.4152
Epoch 2/2
6/6 [==============================] - 4s 740ms/step - loss: 1.0541 - accuracy: 0.4448 - val_loss: 1.0490 - val_accuracy: 0.4152


In [33]:
predictions = model.predict(X_test_embed)
print(history.history)

21/21 [==============================] - 3s 55ms/step
{'loss': [1.0870935916900635, 1.0541021823883057], 'accuracy': [0.4246268570423126, 0.44477611780166626], 'val_loss': [1.0670653581619263, 1.0489977598190308], 'val_accuracy': [0.4151515066623688, 0.4151515066623688]}
